# Bedrock Knowledge Base Retrieval and Generation with Guardrails

## Description
This notebook demonstrates how to enhance a Retrieval-Augmented Generation (RAG) pipeline using Amazon Bedrock with Guardrails for better model control and filtering. We will walk through retrieving data from a knowledge base, applying guardrails to control the generation of responses, and filtering results using metadata.

![Guardrails](./guardrail.png)

## 1: Import Required Functions

In [ ]:
# Import necessary functions from advanced_rag_utils
from advanced_rag_utils import (
    load_variables,
    get_model_arn,
    get_guardrail_arn,
    setup_bedrock_client,
    create_standard_filter,
    retrieve_and_generate_with_conditional_guardrails,
    display_markdown_result,
    extract_text_response
)

## 2: Load Configuration Variables

In [ ]:
# Load the configuration variables
variables = load_variables("../variables.json")
variables

## 3: Define ARN and Configuration Details

In [ ]:
# Setting up configuration for Bedrock and Guardrails
account_number = variables['accountNumber']
knowledge_base_id = variables['kbFixedChunk']   
model_id = 'us.amazon.nova-pro-v1:0' 
guardrail_version = variables['guardrail_version'] 
guardrail_id = variables['guardrail_id']    

# Get ARNs for the model and guardrails using utility functions
model_arn = get_model_arn(account_number, model_id)
guardrail_arn = get_guardrail_arn(account_number, guardrail_id)

## 4: Set Up Bedrock Client

In [ ]:
# Initialize the Bedrock agent runtime client
bedrock_client = setup_bedrock_client("us-west-2")

## 5: Define Metadata Filter

In [ ]:
# Create a standard filter for document type and year
metadata_filter = create_standard_filter('10K Report', 2023)

## 6: Test Guardrails for Investment Advice
Let's ask the model for investment advice. When we created the guardrails, we restricted Bedrock from providing any investment advice. Bedrock should return a preconfigured response "This request cannot be processed due to safety protocols"

In [ ]:
# Define the query that will be sent to the model
query = "based on your amazon's results should I buy amazon stock?"

### 6.1: Response Without Guardrails

In [ ]:
# Get response without applying guardrails
response_without_guardrails = retrieve_and_generate_with_conditional_guardrails(
    query=query, 
    knowledge_base_id=knowledge_base_id, 
    model_arn=model_arn,
    bedrock_client=bedrock_client,
    metadata_filter=metadata_filter,
    use_guardrails=False
)

# Display the response in Markdown format
display_markdown_result(response_without_guardrails)

### 6.2: Response With Guardrails

In [ ]:
# Get response with guardrails applied
response_with_guardrails = retrieve_and_generate_with_conditional_guardrails(
    query=query, 
    knowledge_base_id=knowledge_base_id, 
    model_arn=model_arn,
    bedrock_client=bedrock_client,
    metadata_filter=metadata_filter,
    use_guardrails=True,
    guardrail_id=guardrail_id,
    guardrail_version=guardrail_version
)

# Extract and print the text response
print(extract_text_response(response_with_guardrails))

## 7. Test Guardrails for PII Data

In [ ]:
# Define a new query about Amazon's CFO
query="Who is the current CFO of Amazon?"

### 7.1: Response Without Guardrails

In [ ]:
# Get response without applying guardrails
response_without_guardrails = retrieve_and_generate_with_conditional_guardrails(
    query=query, 
    knowledge_base_id=knowledge_base_id, 
    model_arn=model_arn,
    bedrock_client=bedrock_client,
    metadata_filter=metadata_filter,
    use_guardrails=False
)

# Extract and print the text response
print(extract_text_response(response_without_guardrails))

### 7.2: Response With Guardrails (PII Anonymization)

In [ ]:
# Get response with guardrails applied
response_with_guardrails = retrieve_and_generate_with_conditional_guardrails(
    query=query, 
    knowledge_base_id=knowledge_base_id, 
    model_arn=model_arn,
    bedrock_client=bedrock_client,
    metadata_filter=metadata_filter,
    use_guardrails=True,
    guardrail_id=guardrail_id,
    guardrail_version=guardrail_version
)

# Extract and print the text response
print(extract_text_response(response_with_guardrails))